In [1]:
import os 
import numpy as np
import pandas as pd
import scipy.io
import seaborn as sns

from neo.core import SpikeTrain
from quantities import ms, s, Hz
from elephant.statistics import mean_firing_rate
from elephant.statistics import time_histogram, instantaneous_rate
from elephant.kernels import GaussianKernel


C:\Users\David\Anaconda3\envs\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

for monkey_directory in ['Speriatus', 'Mojo']:
    path = 'C:\\Users\\David\\Desktop\\IDIBAPS\\Gottlib_data\\data files\\distractor paper data only\\' +monkey_directory
    ### path = 'C:\\Users\\David\\Desktop\\fast_process_gottlieb\\' +monkey_directory
    Monkey = path.split('\\')[-1]
    for file_to_use in os.listdir(path):  #
        neuron = file_to_use.split('.')[0]
        print(Monkey, neuron)
        #########
        ### Open file
        f = scipy.io.loadmat(os.path.join(path, file_to_use))
        ########
        ### Check if there is spiking data inside
        if np.shape(f['data']['spikes'][0][0][0]) ==(0,): 
            print('No data')
            
        else:
            ########
            ### Spikes 
            df_spike_time=pd.DataFrame(f['data']['spikes'][0][0][0][0][:, :])
            df_spike_time.columns = [str(i) for i in range(0, np.shape(df_spike_time)[1])]
            ########
            ### Events
            Dict_events={}
            for i in range(0, len(f['data']['events'][0][0][0])     ):
                events = pd.DataFrame( f['data']['events'][0][0][0][i])
                events.columns=['time', 'code']
                Dict_events[str(i)]= events
            #
            ########
            ### Descriptors & Bad trials
            ## Use the Descriptors to make the booleans to select the HIT trials
            lists=[]
            for T in range(0, len(f['data']['descriptors'][0][0][0])):
                lists.append( [f['data']['descriptors'][0][0][0][T][i][0] for i in range(len(f['data']['descriptors'][0][0][0][T]) )] )
            #
            Descriptors = pd.DataFrame(lists)
            Descriptors=Descriptors.transpose()
            Descriptors.columns = [str(i) for i in range(0, np.shape(Descriptors)[1])]
            # 6 "failed" // performance code, as defined below 
            boolean_hit_trials = Descriptors.iloc[6].isin([1]) #Descriptors.iloc[6,:]==1
            ########
            ### Bad trials
            ## take off the bad trials
            bad = pd.DataFrame(np.array([f['data']['bad'][0][0][0][x][0][0] for x in range(0, len(f['data']['bad'][0][0][0]))]))
            bad = bad.transpose()
            bad.columns = [str(i) for i in range(0, np.shape(df_spike_time)[1])]
            #each column in a trial; if 1, discard it
            boolean_bad_trials = bad.iloc[0,:]!=1
            ########
            ########
            ### Boolean combinging success trials and correct
            boolean_keep = boolean_hit_trials & boolean_bad_trials 
            print('Trials keept = ' +str(sum(boolean_keep)))
            print('Trials excluded = ' + str(np.shape(df_spike_time)[1] - sum(boolean_keep) ))
            ########
            ### df_spikes_correct
            indexes_trials_keep = np.array([str(i) for i in range(0, np.shape(df_spike_time)[1])])[boolean_keep]
            df_spikes = df_spike_time[list(indexes_trials_keep)]
            df_spikes = df_spikes.iloc[4:, :] #remove the first 4 rows, no spikes
            df_spikes = df_spikes.transpose()[df_spikes.iloc[0,:]<9999].transpose() #remove columns full of nans
            ########
            ########
            ########
            ### Get the firing rate of each trial (Gaussian fit and auto)
            ### turorial https://elephant.readthedocs.io/en/latest/tutorials/statistics.html
            trials_=[]            
            Descriptors_use = Descriptors[list(df_spikes.columns)]           
            for TRIAL in list(df_spikes.columns):
                ###### firing rates
                times_spikes_all = df_spikes[TRIAL].values # get trial spike times
                times_spikes = times_spikes_all[times_spikes_all<9999] ##remove nans in the trial (at the end)
                times_spikes = times_spikes[times_spikes>0] #no negative timings for spikes
                stop_time =  times_spikes.max()
                spiketrain1 = SpikeTrain(times_spikes, units=ms, t_stop=stop_time) #spiketrain
                histogram_rate = time_histogram([spiketrain1], 10*ms, output='rate') #sampling rate of 10 Suzuki & Gottlieb, increase for less temporal resolution. 
                #inst_rate = instantaneous_rate(spiketrain1, sampling_period=10*ms) 
                gaus_rate = instantaneous_rate(spiketrain1, sampling_period=10*ms, kernel=GaussianKernel(15*ms)) #s.d of Suzuki & Gottlieb 
                times_ = gaus_rate.times.rescale(ms)
                #firing_auto = inst_rate.rescale(histogram_rate.dimensionality).magnitude.flatten()
                firing_gauss = gaus_rate.rescale(histogram_rate.dimensionality).magnitude.flatten()
                df_trial = pd.DataFrame({'times':times_, 'firing_gauss':firing_gauss}) #, 'firing_auto': firing_auto})
                                
                ####### descriptive data
                df_trial['trial']=TRIAL 
                df_trial['TDOA'] = Descriptors_use[TRIAL].iloc[20]
                df_trial['TDconf'] = Descriptors_use[TRIAL].iloc[24]
                df_trial['targetloc'] = Descriptors_use[TRIAL].iloc[1]
                df_trial['distLoc'] = Descriptors_use[TRIAL].iloc[19]
                df_trial['fixationtime'] = Descriptors_use[TRIAL].iloc[9]
                df_trial['failed'] = Descriptors_use[TRIAL].iloc[6]
                df_trial['saccadeChoice'] = Descriptors_use[TRIAL].iloc[18]
                df_trial['cueDuration'] = Descriptors_use[TRIAL].iloc[10]
                df_trial['distDur'] = Descriptors_use[TRIAL].iloc[21]
                df_trial['monkey'] = Monkey
                df_trial['neuron'] = neuron           
                trials_.append(df_trial)
            #
            df_neuron=pd.concat(trials_)
            ########
            ########
            ### Normalization
            # Get the mean of each time in the control condition (T in RF no dist), then get the max of this means
            max_fr_gauss_normalize = df_neuron.loc[(df_neuron['TDconf']==0), ['firing_gauss', 'times']].groupby('times').describe()['firing_gauss']['mean'].max()
            #max_fr_auto_normalize = df_neuron.loc[(df_neuron['TDconf']==0), ['firing_auto', 'times']].groupby('times').describe()['firing_auto']['mean'].max()
            df_neuron['norm_firing_gauss'] = df_neuron['firing_gauss']/max_fr_gauss_normalize
            #df_neuron['norm_firing_auto'] = df_neuron['firing_auto']/max_fr_auto_normalize
            ########
            ########
            ######## Save the dataframe as an excel with the name of the neuron
            path_save = 'C:\\Users\\David\\Desktop\\IDIBAPS\\Gottlib_data\\firing_rates'
            path_save_neuron = os.path.join(path_save, neuron+'.xlsx')
            df_neuron.to_excel(path_save_neuron)




Speriatus 189139306
Trials keept = 313
Trials excluded = 315


C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in less
C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:495: UserWarning: Binning discarded 1 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))
C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:495: UserWarning: Binning discarded 2 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))
C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:990: UserWarning: Correcting 1 rounding errors by shifting the affected spikes into the following bin. You can set tolerance=None to disable this behaviour.
  'behaviour.'.format(num_rounding_corrections))
C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:274: UserWarning: Correcting a rounding error in the calculation of n_bins by increasing n_bins by 1. You 

Speriatus 1891406
Trials keept = 463
Trials excluded = 63
Speriatus 1891407
Trials keept = 368
Trials excluded = 190
Speriatus 189141804
Trials keept = 229
Trials excluded = 121
Speriatus 1891421
Trials keept = 438
Trials excluded = 96


C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:495: UserWarning: Binning discarded 7 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))


Speriatus 1891436
Trials keept = 436
Trials excluded = 130


...

Speriatus 189160702
Trials keept = 405
Trials excluded = 143
Speriatus 189161604
Trials keept = 384
Trials excluded = 176
Speriatus 1891626
Trials keept = 435
Trials excluded = 85
Speriatus 189163101
Trials keept = 421
Trials excluded = 125
Speriatus 189163701
Trials keept = 425
Trials excluded = 155
Speriatus 189164103
Trials keept = 435
Trials excluded = 111
Speriatus 189164404
Trials keept = 442
Trials excluded = 94
Speriatus 189164801
Trials keept = 435
Trials excluded = 109
Speriatus 189164803
Trials keept = 435
Trials excluded = 109
Speriatus 189165003
Trials keept = 444
Trials excluded = 58
Speriatus 1891653
Trials keept = 446
Trials excluded = 222
Speriatus 189168701
Trials keept = 408
Trials excluded = 72
Speriatus 189169502
Trials keept = 437
Trials excluded = 75
Speriatus 189171003
Trials keept = 443
Trials excluded = 77
Speriatus 189171602
Trials keept = 415
Trials excluded = 213
Speriatus 189172801
Trials keept = 460
Trials excluded = 102
Speriatus 189173101
Trials keept =

C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:990: UserWarning: Correcting 4 rounding errors by shifting the affected spikes into the following bin. You can set tolerance=None to disable this behaviour.
  'behaviour.'.format(num_rounding_corrections))


Speriatus 1891783
Trials keept = 475
Trials excluded = 95
Speriatus 189179001
Trials keept = 485
Trials excluded = 55
Speriatus 189179002
Trials keept = 485
Trials excluded = 55
Speriatus 1891793
Trials keept = 495
Trials excluded = 45
Speriatus 189179702
Trials keept = 429
Trials excluded = 94
Speriatus 189180001
Trials keept = 447
Trials excluded = 103
Speriatus 189181001
Trials keept = 485
Trials excluded = 83
Speriatus 189181502
Trials keept = 481
Trials excluded = 139
Speriatus 1891819
Trials keept = 442
Trials excluded = 94
Speriatus 189182202
Trials keept = 458
Trials excluded = 86
Speriatus 189182501
Trials keept = 483
Trials excluded = 137
Speriatus 189182901
Trials keept = 479
Trials excluded = 159
Speriatus 189183201
Trials keept = 466
Trials excluded = 130
Speriatus 189183402
Trials keept = 472
Trials excluded = 256
Speriatus 1891841
Trials keept = 450
Trials excluded = 198
Speriatus 189184601
Trials keept = 456
Trials excluded = 162
Speriatus 1891857
Trials keept = 410
Tri

C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:495: UserWarning: Binning discarded 10 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))
C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:495: UserWarning: Binning discarded 9 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))
C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:495: UserWarning: Binning discarded 8 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))
C:\Users\David\AppData\Roaming\Python\Python35\site-packages\elephant\conversion.py:495: UserWarning: Binning discarded 11 last spike(s) in the input spiketrain.
  n=n_spikes - n_spikes_binned))


Speriatus 189194301
Trials keept = 422
Trials excluded = 256
Speriatus 189195301
Trials keept = 485
Trials excluded = 75
Speriatus 189196301
Trials keept = 174
Trials excluded = 58
Speriatus 189196401
Trials keept = 276
Trials excluded = 184
Speriatus 1891976
No data
Speriatus 1891980
No data
Speriatus 1891983
No data
Speriatus 1891986
No data
Speriatus 1891994
No data
Speriatus 1891997
No data
Speriatus 1892003
No data
Speriatus 1892009
No data
Speriatus 1892015
No data
Speriatus 1892018
No data
Speriatus 1892063
Trials keept = 587
Trials excluded = 201
Speriatus 189207101
Trials keept = 588
Trials excluded = 344
Speriatus 189207501
Trials keept = 296
Trials excluded = 194
Speriatus 189208001
Trials keept = 595
Trials excluded = 587
Speriatus 189209101
Trials keept = 537
Trials excluded = 218
Speriatus 189209503
Trials keept = 512
Trials excluded = 150
Speriatus 189210001
Trials keept = 291
Trials excluded = 173
Speriatus 189210901
Trials keept = 448
Trials excluded = 114
Speriatus 18